In [ ]:
!pip install datasets accelerate peft bitsandbytes trl==0.8.6 flash-attn

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline
import torch

base_model_id = "microsoft/Phi-3-mini-4k-instruct"
groundedai_eval_id = "grounded-ai/phi3-hallucination-judge"

config = PeftConfig.from_pretrained(groundedai_eval_id)
base_model = AutoModelForCausalLM.from_pretrained(base_model_id)
model_peft = PeftModel.from_pretrained(base_model, groundedai_eval_id, config=config)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

merged_model = model_peft.merge_and_unload()
merged_model.to('cuda')

In [ ]:
def format_func(query, response):
    prompt = f"""Your job is to evaluate whether a machine learning model has hallucinated or not.
    A hallucination occurs when the response is coherent but factually incorrect or nonsensical
    outputs that are not grounded in the provided context.
    You are given the following information:
      ####INFO####
      [Query]: {query}
      [Model Response]: {response}
      ####END INFO####
      Based on the information provided is the model output a hallucination? Respond with only "yes" or "no"
      """
    return prompt

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

pipe = pipeline(
    "text-generation",
    model=merged_model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 256,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 yes


In [ ]:
def run_merged_model(query, response):
  input = format_func(query, response)
  messages = [
      {"role": "user", "content": input}
  ]

  pipe = pipeline(
      "text-generation",
      model=merged_model,
      tokenizer=tokenizer,
  )

  generation_args = {
      "max_new_tokens": 2,
      "return_full_text": False,
      "temperature": 0.01,
      "do_sample": True,
  }

  output = pipe(messages, **generation_args)
  torch.cuda.empty_cache()
  return output[0]['generated_text'].strip().lower()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Test for Hallucination

In [ ]:
run_merged_model(query='Based on the follwoing <context>Walrus are the largest mammal</context> answer the question <query> What is the best PC?</query>',response='The best PC is the mac')

'yes'